In [1]:
import gc

import numpy as np
import torch

/home/rpc/Desktop/RecSystems/Project/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Cuda maintenance
gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Torch device: ", device)


Torch device:  cuda


In [3]:
import pandas as pd

In [4]:
data_path = '/home/rpc/Desktop/RecSystems/Project/data'

In [5]:
!find {data_path}

/home/rpc/Desktop/RecSystems/Project/data
/home/rpc/Desktop/RecSystems/Project/data/чековые данные.csv
/home/rpc/Desktop/RecSystems/Project/data/Product_dict.pkl
/home/rpc/Desktop/RecSystems/Project/data/.~lock.чековые данные.csv#
/home/rpc/Desktop/RecSystems/Project/data/products.csv


In [6]:
receipts_data = pd.read_csv("/home/rpc/Desktop/RecSystems/Project/data/чековые данные.csv",
                            parse_dates=['sale_date_date'],
                            on_bad_lines='skip')

/tmp/ipykernel_5998/1885024923.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  receipts_data = pd.read_csv("/home/rpc/Desktop/RecSystems/Project/data/чековые данные.csv",


In [7]:
receipts_data.head()

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,(197309) Пакет 28см х 50см,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,(197310) Пакет 30см х 60см,906.0,205.0,-1.0,"1,00"


In [8]:
receipts_data.shape

(20000003, 9)

In [9]:
receipts_data = receipts_data.dropna()

In [10]:
receipts_data.shape

(19989262, 9)

In [11]:
receipts_data[["contact_id", "shop_id", "product_id"]] =\
    receipts_data[["contact_id", "shop_id", "product_id"]].apply(pd.to_numeric, errors='coerce', downcast='integer')

In [12]:
receipts_data.head(3)

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455,168308,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00"


In [13]:
receipts_data = receipts_data.dropna()

In [14]:
receipts_data.shape

(19989262, 9)

In [15]:
receipts_data = receipts_data.drop_duplicates()

In [16]:
receipts_data.shape

(19703842, 9)

In [17]:
# Add user ids

contact_ids = receipts_data.contact_id.unique()

In [18]:
contact_ids_map = {contact_ids[i]: i for i in range(len(contact_ids))}

In [19]:
product_ids = receipts_data.product_id.unique()
product_ids_map = {product_ids[i]: i for i in range(len(product_ids))}

In [20]:
receipts_data["contact_serial_number"] = receipts_data["contact_id"].apply(lambda x: contact_ids_map[x]) 

In [21]:
receipts_data["product_serial_number"] = receipts_data["product_id"].apply(lambda x: product_ids_map[x]) 

In [23]:
receipts_data.shape

(19703842, 11)

In [24]:
#receipts_data.to_csv(index=False)

compression_opts = dict(method='zip', archive_name='prefiltered.csv')  
receipts_data.to_csv('prefiltered.zip', index=False, compression=compression_opts)  

In [26]:
receipts_data_filtered = receipts_data.loc[~receipts_data['product_id'].isin(bags_ids)]

In [27]:
receipts_data_filtered.shape

(19449494, 11)

### Split to train and test

----

In [59]:
from surprise import Reader
from surprise import Dataset

In [60]:
# https://implicit.readthedocs.io/en/latest/als.html
# https://medium.com/@patelneha1495/recommendation-system-in-python-using-als-algorithm-and-apache-spark-27aca08eaab3
# https://surprise.readthedocs.io/en/stable/prediction_algorithms.html
# https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b
# https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Building%20Recommender%20System%20with%20Surprise.ipynb

reader = Reader(rating_scale=(0, 1))
data = +.load_from_df(receipts_data_filtered[['contact_serial_number', 'product_serial_number']], reader)

ValueError: not enough values to unpack (expected 3, got 2)

In [18]:
product_data = pd.read_csv("/home/rpc/Desktop/RecSystems/Project/data/products.csv")

In [19]:
product_data.head()

,product_id,product
0,168308,(197312) Пакет-майка 25см х 45см 906
1,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...
2,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...
3,168570,(197309) Пакет 28см х 50см 906
4,146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906"


In [20]:
def find_product_by_description(value, description):
    return description in value
    
new_df = product_data[product_data.apply(lambda x : find_product_by_description(x['product'], "Пакет"),axis=1)]

In [32]:
new_df[70:80]#.product_id.values

,product_id,product
27163,84067,"(58270) Пакет ""Майка"" 16х30 906"
27598,142381,(117095) Пакет VIP белый 23х20х10см картоновый...
28007,214210,(200068) Пакет упак.прозр.с голографич.рисунко...
29787,111189,(36839) Пакет перевяз. индив. стерильный -1
29905,165941,(194129) ПП Пакет 25 х 30см 906
29955,91825,"(17671) Пакет перевяз. 1 подушечка, эластичный..."
30133,96059,(117863) РЧК Пакет подарочный Венеция 31х23см 906


In [25]:
bags_ids = set([168308, 168570, 146960, 94270, 168319, 31222, 22832, 105675, 164928,
           62164, 122035, 107261, 105192, 56633, 216807, 90961, 160777, 160785, 100851,
           153121, 57815, 160496, 350140, 85927, 53949, 26647,
           168280,  41489, 339113, 161366, 156035, 156034,  45627,  91501,
           358775, 166825, 216816, 107898, 214400, 358718, 217507, 223330, 118208,  94155,
           37790, 168471, 234857, 234856,  41435, 117406, 222884, 222902,  76673, 138589,
           358733, 222711, 84067, 142381, 214210, 165941,  96059])

In [10]:
import pickle

with open('/home/rpc/Desktop/RecSystems/Project/data/Product_dict.pkl', 'rb') as f:
    product_dict = pickle.load(f)

In [11]:
count = 0
for item in product_dict.items():
    if count > 10:
        break
    print(item)
    count += 1

('168308', '(197312) Пакет-майка 25см х 45см 906')
('134832', '(62448) Перекись водорода р-р наружн. 3% фл.полимерн. 100мл 404')
('101384', '(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404')
('168570', '(197309) Пакет 28см х 50см 906')
('146960', '(111023) Пакет "Аптека Озерки" 28 х 35см 906')
('128700', '(51806) Ранитидин тб 150мг уп N20 595')
('52539', '(19416) Платки носовые "Зева Кидс" №10 721')
('94270', '(66684) Пакет майка "Благодарим за покупку" (28+14) х 43см №1 -1')
('77426', '(5605) Иглы Микро-Файн д/шприц-ручки 30G 0,3*8мм N100 759')
('92475', '(114214) Дорзопт капли глазные 2% 5мл. фл.-кап. N3 575')
('158643', '(181230) PL Шприц одноразовый 3-комп. 3мл №1 762')


In [12]:
# Add users and items sequence number

receipts_data.contact_id.unique()

array([1260627, 198287, 2418385, ..., '1788779', nan, 'к: 20000000'],
      dtype=object)

### Plan


---

Постановка задачи: разработать рекомендательную систему для аптек. Представьте, что вы приходите на кассу, а у кассира во время формирования чека всплывает подсказка, что можно вам ещё порекомендовать.

---


### Preprocessing

---

* Delete NaNs
* Delete users who bought above average (оптовые) ... build hists
* 

#### Model 1

**Кандидатогенерация** (два разных подхода):
1. Build matrix of users and items and apply SVD or ALS from ``surprise`` lib
2. Using word2vec calculate numerical representation from product description and using cos dist or KNN select recommendations 

Результаты можно смешать

**Ранжирование**:

---

Алгоритм для каждого item’а отдаёт значения оценок, по которым их можно отсортировать

---

2. Using word2vec calculate numerical representation from selected products description and using cos dist or KNN range recommendations

#### Model 2

Additional features:
1. User bought already this product
2. Seasonal products, popular products
...

Build binary classificator based on products and users features.



How to build datasets:
Validation:
1. Group items into one receipt (list of bought products by one user) by sale_date_date, contact_id and shop_id
2. Mark part of this list as input, generate recommendations based on this input and compare recommendations with the remaining part of the list. Calculate metric:

$$ \frac{количество купленных товаров из рекомендованного}{N} $$






If user didn't bought a popular item, then it's a TN


#### Additional Option

Use LightFM lib for prediction. In my case I have a problem with LightFM installation: https://github.com/lyst/lightfm/issues/671